In [1]:
import os
from datetime import datetime

from core.runner import Runner
from core.environment import SingleVertexSpinFoam, StarModelSpinFoam


def train_gfn(
    spin_j,
    sf_model,
    main_layer_hidden_nodes,
    branch1_hidden_nodes,
    branch2_hidden_nodes,
    activation,
    exploration_rate,
    training_fraction_from_back_traj,
    learning_rate,
    batch_size,
    n_iterations,
    evaluation_batch_size,
    generate_samples_every_m_training_samples,
):
    if sf_model == "single_vertex_model":
        spinfoam_model = SingleVertexSpinFoam(spin_j=spin_j)
    elif sf_model == "star_model":
        spinfoam_model = StarModelSpinFoam(spin_j=spin_j)
    else:
        raise ValueError(
            "Spinfoam model not yet implemented. " "Custom Spinfoam class can be made."
        )

    # training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")

    directory_for_generated_samples = f"./data/GFlowNet/{sf_model}/j_{spin_j}/n_iterations_{n_iterations}/main_layer_hid_nodes_{main_layer_hidden_nodes}/exploration_rate_{exploration_rate}/learning_rate_{learning_rate}/batch_size_{batch_size}"

    runner = Runner(
        spinfoam_model=spinfoam_model,
        main_layer_hidden_nodes=main_layer_hidden_nodes,
        branch1_hidden_nodes=branch1_hidden_nodes,
        branch2_hidden_nodes=branch2_hidden_nodes,
        activation=activation,
        exploration_rate=exploration_rate,
        training_fraction_from_back_traj=training_fraction_from_back_traj,
        learning_rate=learning_rate,
    )

    ave_losses = runner.train_agent(
        batch_size,
        n_iterations,
        evaluation_batch_size,
        generate_samples_every_m_training_samples,
        directory_for_generated_samples,
    )

    return ave_losses

2023-05-10 11:21:55.728264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 11:21:55.845506: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-10 11:21:56.331782: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/frisus/Scrivania/sl2cfoam-next-dev/lib:/home/frisus/anaconda3/envs/tf/lib/
2023-05-10 11:21:56.331831: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

## Vertex Amplitude

In [4]:
spin_j = 2.0
sf_model = "star_model"
main_layer_hidden_nodes = (30, 20)
branch1_hidden_nodes = ()
branch2_hidden_nodes = ()
activation = "swish"
exploration_rate = 0.5
training_fraction_from_back_traj = 0.0
learning_rate = 0.0005
batch_size = int(5e2)
n_iterations = int(5e3)
evaluation_batch_size = int(5e4)
generate_samples_every_m_training_samples = int(5e4)

In [5]:
ave_losses = train_gfn(
    spin_j,
    sf_model,
    main_layer_hidden_nodes,
    branch1_hidden_nodes,
    branch2_hidden_nodes,
    activation,
    exploration_rate,
    training_fraction_from_back_traj,
    learning_rate,
    batch_size,
    n_iterations,
    evaluation_batch_size,
    generate_samples_every_m_training_samples,
)

Nth iteration: 100 Trained on K samples: 50000 Average Loss: 64.422575157154967
Nth iteration: 200 Trained on K samples: 100000 Average Loss: 35.141102826201035
Nth iteration: 300 Trained on K samples: 150000 Average Loss: 31.566294092764487
Nth iteration: 400 Trained on K samples: 200000 Average Loss: 25.880135097169664
Nth iteration: 500 Trained on K samples: 250000 Average Loss: 22.92952863419351
Nth iteration: 600 Trained on K samples: 300000 Average Loss: 21.485759464330496
Nth iteration: 700 Trained on K samples: 350000 Average Loss: 20.267821919020633
Nth iteration: 800 Trained on K samples: 400000 Average Loss: 20.889253686118447
Nth iteration: 900 Trained on K samples: 450000 Average Loss: 21.637274807374204
Nth iteration: 1000 Trained on K samples: 500000 Average Loss: 17.521728326780771
Nth iteration: 1100 Trained on K samples: 550000 Average Loss: 18.457537865879377
Nth iteration: 1200 Trained on K samples: 600000 Average Loss: 18.39601951616125
Nth iteration: 1300 Trained 